# Rock-Paper-Scissors CNN Classification from Image

Ahmad Agriansyah | email: ahmadagriansyah1@gmail.com

# Import Library

In [4]:
import numpy as np, matplotlib.pyplot as plt, matplotlib.image as mpimg, tensorflow as tf, os
from google.colab import files
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

# Directory

In [5]:
os.listdir('/content/drive/MyDrive/Colab Notebooks/Rock Paper Scissor')

['rockpaperscissors.ipynb', '__MACOSX', 'rockpaperscissors']

In [6]:
## Base Directory ##
base_dir = '/content/drive/MyDrive/Colab Notebooks/Rock Paper Scissor/rockpaperscissors/rps-cv-images'

In [7]:
## Data Container ##
rock = os.path.join('/content/drive/MyDrive/Colab Notebooks/Rock Paper Scissor/rockpaperscissors/rock')
paper = os.path.join('/content/drive/MyDrive/Colab Notebooks/Rock Paper Scissor/rockpaperscissors/paper')
scissors = os.path.join('/content/drive/MyDrive/Colab Notebooks/Rock Paper Scissor/rockpaperscissors/scissors')

In [8]:
## Cek Jumlah Data ##
training_rock = os.listdir(rock)
training_paper = os.listdir(paper)
training_scissors = os.listdir(scissors)

print('Rock :', len(training_rock))
print('Paper :', len(training_paper))
print('Scissors :', len(training_scissors))

Rock : 726
Paper : 712
Scissors : 750


#Pre-processing

In [9]:
## Rescale, rotate, shear angle, split dataset 40% ##
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, horizontal_flip=True,
                                   shear_range=0.2, fill_mode='wrap', validation_split=0.4)

train_datagen

In [10]:
## Targeting Data + Shuffle ##
train_gen = train_datagen.flow_from_directory(base_dir, target_size=(100,150),
                                              shuffle=True, class_mode='categorical',
                                              subset='training')
validation_gen = train_datagen.flow_from_directory(base_dir, target_size=(100,150),
                                              shuffle=True, class_mode='categorical',
                                              subset='validation')

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


# Showing Images

In [ ]:
## Formatted image 5x5 ##
nrows = 5
ncols = 5

## Iteration index ##
pic_index = 0

fig = plt.gcf()
fig.set_size_inches(ncols * 5, nrows * 5)

pic_index += 8
next_rock_pix = [os.path.join(rock, fname)
                 for fname in training_rock[pic_index - 8:pic_index]]
next_paper_pix = [os.path.join(paper, fname)
                  for fname in training_paper[pic_index - 8:pic_index]]
next_scissors_pix = [os.path.join(scissors, fname)
                     for fname in training_scissors[pic_index - 8:pic_index]]

for i, img_path in enumerate(next_rock_pix + next_paper_pix + next_scissors_pix):
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off')

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

# CNN Architecture

In [ ]:
model = tf.keras.models.Sequential([
    ## First Layer ##
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(100,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    ## Second Layer ##
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    ## Third Layer ##
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    ## Fourth Layer ##
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    ## Flatten + Dense ##
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 148, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 17, 128)       0

# Compiling

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

# Stop callback threshold

In [ ]:
accuracy_threshold = 98e-2
class model_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') >= accuracy_threshold:
            print('\nFor Epoch', epoch,
                  '\nAccuracy has reached %2.2f%%' % (logs['accuracy'] * 100),
                  ', training has been stopped')
            self.model.stop_training = True

# Model Training

In [ ]:
history = model.fit(train_gen, steps_per_epoch=25, epochs=20,
                    validation_data=validation_gen,
                    validation_steps=5, verbose=2,
                    callbacks=[model_callback()])

In [3]:
## Upload Image ##
upl_img = files.upload()

for fn in upl_img.keys():
  path = fn
  img = image.load_img(path, target_size=(100,150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  plt.axis('Off')
  plt.imshow(img)
  plt.show()

  ## Menampilkan class ##

  print(fn)
  if classes[0,0]==1:
    print('Image Classifier: Paper')
  elif classes[0,1]==1:
    print('Image Classifier: Rock')
  elif classes[0,2]==1:
    print('Image Classifier: Scissors')
  else:
    print('Image Classifier: None')

Saving 1MOm36DHK0R8OfIC.png to 1MOm36DHK0R8OfIC.png


NameError: ignored